1.针对fashion_mnist数据集，设计一个CNN网络，训练fashion_mnist的分类操作，将准确率提高到92%以上！
请尝试通过优化网络层数与超参数、正则化等措施，提高准确率。
请与第二周MLP模型相比较，在文末总结说明不同模型在收敛速度与分类准确率的表现。
注意：输入数据需要有Channel维度，需要reshape为(28,28,1)

In [6]:
from tensorflow import keras
from keras.utils import np_utils
import os

from keras.datasets import fashion_mnist
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()
train_images.shape, train_labels.shape

((60000, 28, 28), (60000,))

In [7]:
import numpy as np
# flip_temp = np.swapaxes()
flip_temp = train_images.transpose((0,2,1))
train_images = np.concatenate((train_images,flip_temp),axis=0)
train_labels = np.concatenate((train_labels,train_labels),axis=0)
train_images.shape, train_labels.shape

((120000, 28, 28), (120000,))

In [8]:
x_train = train_images.reshape(-1,28,28,1).astype('float32')
x_test = test_images.reshape(-1,28,28,1).astype('float32')
x_train_norm = x_train / 255
x_test_norm = x_test / 255
y_train_onehot = np_utils.to_categorical(train_labels)
y_test_onehot = np_utils.to_categorical(test_labels)

In [9]:
print(x_train_norm.shape, y_train_onehot.shape)

(120000, 28, 28, 1) (120000, 10)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D

model = Sequential()

model.add(Conv2D(filters=16,kernel_size=(5,5),input_shape=(28,28,1),
                 activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(4,4),
                 activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),
                 activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(rate=0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(10,activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x=x_train_norm, y=y_train_onehot,
                    validation_split=0.2, epochs=30,
                    batch_size=200, verbose=2)

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(history, train, validation):
    plt.plot(history.history[train])
    plt.plot(history.history[validation])
    plt.title('train history')
    plt.ylabel(train)
    plt.xlabel('epoch')
    plt.legend(['train','validation'], loc='upper left')
    plt.grid()
    plt.show()

show_train_history(history,'accuracy','val_accuracy')
show_train_history(history,'loss','val_loss')

In [ ]:
scores = model.evaluate(x_test_norm, y_test_onehot)
print()
print('accuracy=', scores[1])
print(scores)

In [ ]:
prediction = np.argmax(model.predict(x_test_norm), axis=-1)
prediction

In [ ]:
mis_predicted = []
for i in range(prediction.shape[0]):
    if test_labels[i] != prediction[i]:
        mis_predicted.append(i)
print(mis_predicted)

In [ ]:
def plot_labels_prediction(images, labels, prediction, idx, num=10):
    fig = plt.gcf()
    fig.set_size_inches(12,14)
    if num>25: num=25
    for i in range(0, num):
        ax = plt.subplot(5, 5, i+1)
        ax.imshow(images[idx], cmap='binary')
        title = 'label=' + str(labels[idx])
        if len(prediction)>0:
            title += ',predit=' + str(prediction[idx])
        
        ax.set_title(title, fontsize=10)
        ax.set_xticks([]);ax.set_yticks([])
        idx += 1
    plt.show()

plot_labels_prediction(test_images, test_labels, prediction, idx=110)

In [ ]:
import pandas as pd
pd.crosstab(test_labels, prediction, rownames=['label'], colnames=['predict'])

In [ ]:
plot_labels_prediction(test_images, test_labels, prediction, idx=1665, num=1)